In [57]:
from pycaret.time_series import *
import pandas as pd

df = pd.read_csv("province/data_eng_day.csv", parse_dates=["timestamp"], index_col="timestamp")
df.index = df.index.normalize()
df = df.drop(columns=["Unnamed: 0"])

print(df.index.duplicated().sum())  # นับจำนวน index ที่ซ้ำกัน
print(df.index[df.index.duplicated()])  # แสดงค่าที่ซ้ำ
df = df[~df.index.duplicated(keep="last")]
df = df.asfreq("D")  # ตั้งค่าความถี่เป็นรายวัน
df = df.interpolate()


# แปลงคอลัมน์ timestamp ให้เป็น DatetimeIndex หากยังไม่ได้ทำ
df.index = pd.to_datetime(df.index)
# ตรวจสอบให้แน่ใจว่าไม่มีคอลัมน์ที่ไม่ต้องการ
df = df[['pm_2_5', 'humidity', 'pm_10', 'pm_2_5_sp', 'temperature']]  # ใช้เฉพาะคอลัมน์ที่ต้องการ

# ลองเรียกใช้งาน setup อีกครั้ง
train_size = int(len(df))-7
train_df = df.iloc[:train_size]  # ใช้ข้อมูลก่อนหน้าทั้งหมดเป็น train set
test_df = df.iloc[train_size:]  
train_df.index = train_df.index.to_period("D")
test_df.index = test_df.index.to_period("D")
print(len(train_df), len(test_df))

1
DatetimeIndex(['2025-02-11'], dtype='datetime64[ns]', name='timestamp', freq=None)
676 7


In [58]:
from pycaret.time_series import TSForecastingExperiment
s = TSForecastingExperiment()


target = 'pm_2_5'
print(train_df["pm_2_5"])

# เรียก setup() โดยใช้ train_df
exp = setup(train_df, target=target, fh=7, session_id=123, fold_strategy='expanding')



timestamp
2023-04-01     4.727273
2023-04-02     9.656243
2023-04-03    14.585213
2023-04-04    11.938022
2023-04-05    10.733566
                ...    
2025-01-31    24.003417
2025-02-01    31.669725
2025-02-02    32.655212
2025-02-03    35.094660
2025-02-04    24.459172
Freq: D, Name: pm_2_5, Length: 676, dtype: float64


,Description,Value
0,session_id,123
1,Target,pm_2_5
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(676, 5)"
5,Transformed data shape,"(676, 5)"
6,Transformed train set shape,"(669, 5)"
7,Transformed test set shape,"(7, 5)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [59]:
df.tail(10)

,pm_2_5,humidity,pm_10,pm_2_5_sp,temperature
timestamp,,,,,
2025-02-02,32.655212,70.341639,37.697595,30.528064,27.683890
2025-02-03,35.094660,65.114261,39.898058,33.734223,28.644544
2025-02-04,24.459172,70.266522,28.523077,21.000000,27.775432
2025-02-05,13.987089,71.500943,16.699531,10.823944,28.327450
2025-02-06,19.454865,69.511783,23.709261,15.794842,28.579177
2025-02-07,29.321219,70.933952,34.743259,26.166471,28.271927
2025-02-08,26.000000,69.348548,30.654249,22.380675,28.385596
2025-02-09,24.616898,69.916384,28.971065,20.976852,28.373742
2025-02-10,25.856981,71.226336,30.652667,22.374574,28.251201


In [5]:
best = create_model("arima")
# arima_model = create_model("auto_arima")

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,18:52:21
Status,. . . . . . . . . . . . . . . . . .,Fitting 3 Folds
Estimator,. . . . . . . . . . . . . . . . . .,ARIMA


KeyboardInterrupt: 

In [60]:
# Step 1: พยากรณ์ Exogenous Variables
features = ['humidity', 'pm_10', 'pm_2_5_sp', 'temperature']  # ฟีเจอร์ที่เราจะพยากรณ์
print(features)
# สร้าง DataFrame สำหรับเก็บค่าพยากรณ์
future_exog = pd.DataFrame(index=test_df.index)

# for feature in features:
#     exp = setup(
#     data=train_df[feature],
#     fh=8,
#     fold = 5,
#     session_id=123
# )
#     # สร้างโมเดล ARIMA สำหรับแต่ละฟีเจอร์
#     model = create_model("auto_arima")
    
#     # พยากรณ์อนาคตของฟีเจอร์แต่ละตัว
#     preds = exp.predict_model(model, fh=len(test_df))
    
#     # เก็บค่าพยากรณ์ใน future_exog
#     future_exog[feature] = preds.values

# # Step 2: ใช้ฟีเจอร์ที่พยากรณ์ได้เพื่อพยากรณ์ pm_2_5
# # # ใช้โมเดล ARIMA สำหรับ 'pm_2_5'
# exp = setup(
#     data=train_df[feature],
#     fh=8,
#     fold = 5,
#     session_id=123
# )
# arima_model = exp.create_model("auto_arima", target='pm_2_5')

# # พยากรณ์ `pm_2_5` โดยใช้ `future_exog` เป็น Exogenous Variables
# future_predictions = exp.predict_model(arima_model, fh=len(test_df), X=future_exog)

# # Step 3: แสดงผลลัพธ์
# print(future_predictions)


# predictions = predict_model(best);
# print(predictions)


['humidity', 'pm_10', 'pm_2_5_sp', 'temperature']


In [61]:
exp = setup(
    data=train_df["humidity"],
    fh=8,
    fold = 5,
    session_id=123,  # ตั้งค่า seed เพื่อให้ผลลัพธ์สม่ำเสมอ
    n_jobs=1
)
    # สร้างโมเดล ARIMA สำหรับแต่ละฟีเจอร์
model = create_model("arima", verbose=False)

# พยากรณ์อนาคตของฟีเจอร์แต่ละตัว
preds = exp.predict_model(model, fh=len(test_df))

# เก็บค่าพยากรณ์ใน future_exog
future_exog["humidity"] = preds.values

,Description,Value
0,session_id,123
1,Target,humidity
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(676, 1)"
5,Transformed data shape,"(676, 1)"
6,Transformed train set shape,"(668, 1)"
7,Transformed test set shape,"(8, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [62]:
exp = setup(
    data=train_df["pm_10"],
    fh=8,
    fold = 5,
    session_id=123,  # ตั้งค่า seed เพื่อให้ผลลัพธ์สม่ำเสมอ
    n_jobs=1  # ปิดการใช้งาน multiprocessing เพื่อป้องกันข้อผิดพลาด
)
    # สร้างโมเดล ARIMA สำหรับแต่ละฟีเจอร์
model = create_model("arima", verbose=False)

# พยากรณ์อนาคตของฟีเจอร์แต่ละตัว
preds = exp.predict_model(model, fh=len(test_df))

# เก็บค่าพยากรณ์ใน future_exog
future_exog["pm_10"] = preds.values

,Description,Value
0,session_id,123
1,Target,pm_10
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(676, 1)"
5,Transformed data shape,"(676, 1)"
6,Transformed train set shape,"(668, 1)"
7,Transformed test set shape,"(8, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [63]:
exp = setup(
    data=train_df["pm_2_5_sp"],
    fh=8,
    fold = 5,
    session_id=123,  # ตั้งค่า seed เพื่อให้ผลลัพธ์สม่ำเสมอ
    n_jobs=1  # ปิดการใช้งาน multiprocessing เพื่อป้องกันข้อผิดพลาด
)
    # สร้างโมเดล ARIMA สำหรับแต่ละฟีเจอร์
model = create_model("arima", verbose=False)

# พยากรณ์อนาคตของฟีเจอร์แต่ละตัว
preds = exp.predict_model(model, fh=len(test_df))

# เก็บค่าพยากรณ์ใน future_exog
future_exog["pm_2_5_sp"] = preds.values

,Description,Value
0,session_id,123
1,Target,pm_2_5_sp
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(676, 1)"
5,Transformed data shape,"(676, 1)"
6,Transformed train set shape,"(668, 1)"
7,Transformed test set shape,"(8, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [64]:
exp = setup(
    data=train_df["temperature"],
    fh=8,
    fold = 5,
    session_id=123,  # ตั้งค่า seed เพื่อให้ผลลัพธ์สม่ำเสมอ
    n_jobs=1  # ปิดการใช้งาน multiprocessing เพื่อป้องกันข้อผิดพลาด
)
    # สร้างโมเดล ARIMA สำหรับแต่ละฟีเจอร์
model = create_model("arima")

# พยากรณ์อนาคตของฟีเจอร์แต่ละตัว
preds = exp.predict_model(model, fh=len(test_df))

# เก็บค่าพยากรณ์ใน future_exog
future_exog["temperature"] = preds.values

,Description,Value
0,session_id,123
1,Target,temperature
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(676, 1)"
5,Transformed data shape,"(676, 1)"
6,Transformed train set shape,"(668, 1)"
7,Transformed test set shape,"(8, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2024-12-18,0.6721,0.6109,0.6480,0.8229,0.0237,0.0233,-1.5430
1,2024-12-26,0.6745,0.5224,0.6477,0.7006,0.0230,0.0228,-0.6324
2,2025-01-03,0.3216,0.2919,0.3086,0.3905,0.0108,0.0107,-0.7394
3,2025-01-11,2.7279,2.0412,2.5998,2.7159,0.1000,0.0948,-10.5529
4,2025-01-19,0.9126,0.7752,0.8740,1.0343,0.0314,0.0320,-0.8644
Mean,NaT,1.0617,0.8483,1.0156,1.1328,0.0378,0.0367,-2.8664
SD,NaT,0.8541,0.6165,0.8124,0.8185,0.0318,0.0298,3.8563


In [65]:


# เรียกใช้ setup() เพื่อเตรียมข้อมูล
exp = setup(
    data=train_df,  # ใช้ train_df
    target='pm_2_5',  # ระบุคอลัมน์ target
    fh=8,  # Forecast horizon
    fold=5,  # จำนวน fold สำหรับ cross-validation
    session_id=123,  # ตั้งค่า seed เพื่อให้ผลลัพธ์สม่ำเสมอ
    n_jobs=1  # ปิดการใช้งาน multiprocessing เพื่อป้องกันข้อผิดพลาด
)

# สร้างโมเดล ARIMA
arima_model = create_model("arima", verbose=False)  # สร้างโมเดล ARIMA

# พยากรณ์อนาคตของ `pm_2_5` โดยใช้ `future_exog` เป็น Exogenous Variables
future_predictions = predict_model(arima_model, fh=len(test_df)+1)

,Description,Value
0,session_id,123
1,Target,pm_2_5
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(676, 5)"
5,Transformed data shape,"(676, 5)"
6,Transformed train set shape,"(668, 5)"
7,Transformed test set shape,"(8, 5)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,ARIMA,0.0606,0.0564,0.4217,0.5266,0.0171,0.0169,0.9879


In [66]:
final_best = finalize_model(arima_model)

In [67]:
future_dates = test_df
print(future_dates)

future_df = pd.DataFrame(future_dates).drop(columns=[target])


               pm_2_5   humidity      pm_10  pm_2_5_sp  temperature
timestamp                                                          
2025-02-05  13.987089  71.500943  16.699531  10.823944    28.327450
2025-02-06  19.454865  69.511783  23.709261  15.794842    28.579177
2025-02-07  29.321219  70.933952  34.743259  26.166471    28.271927
2025-02-08  26.000000  69.348548  30.654249  22.380675    28.385596
2025-02-09  24.616898  69.916384  28.971065  20.976852    28.373742
2025-02-10  25.856981  71.226336  30.652667  22.374574    28.251201
2025-02-11  29.587755  76.329751  33.787755  27.918367    27.633866


In [69]:
print(future_exog)
print("ประเภทดัชนี future_exog:", future_exog.index.dtype)
print("ประเภทดัชนี train_df:", train_df.index.dtype)
print("ประเภทดัชนี test_df:", test_df.index.dtype)
predictions_future = predict_model(final_best,fh=7,X=future_exog)
predictions_future

            humidity    pm_10  pm_2_5_sp  temperature
timestamp                                            
2025-02-05   81.1254  35.2329    27.7259      28.5903
2025-02-06   75.6046  19.6691    13.2119      28.6981
2025-02-07   87.8906  35.3725    27.8286      28.7954
2025-02-08   86.6960  19.7623    13.2783      28.8833
2025-02-09   77.3937  35.4496    27.8824      28.9627
2025-02-10   92.2507  19.8338    13.3278      29.0344
2025-02-11   93.4835  35.5191    27.9304      29.0991
ประเภทดัชนี future_exog: period[D]
ประเภทดัชนี train_df: period[D]
ประเภทดัชนี test_df: period[D]


,y_pred
2025-02-05,30.7483
2025-02-06,16.7307
2025-02-07,30.8298
2025-02-08,16.7502
2025-02-09,30.9485
2025-02-10,16.7802
2025-02-11,30.9219
